In [ ]:
from pyaci import Node, options
import logging
import requests

requests.packages.urllib3.disable_warnings()

# Establish a handle to the APIC, and login.
apic = Node('https://praveek6-bld.insieme.local:7000')
apic.methods.Login('admin', 'ins3965!').POST()

In [ ]:
# Create a local reference to a test tenant, and a test network.
mit = apic.mit
mit.polUni().fvTenant('test').fvCtx('test')

# Dump the XML representation of the local MIT.
print 'XML of the local MIT cache:'
print mit.Xml

# Dump the JSON representation of the local MIT.
print 'JSON of the local MIT cache:'
print mit.Json

In [ ]:
# Commit the local reference/cache to APIC.
mit.POST(format='xml')

In [ ]:
# Fetch the management tenant from APIC.
tenant = apic.mit.polUni().fvTenant('mgmt')
tenant.GET()
print tenant.Xml

In [ ]:
# Fetch the in-band BD subtree from APIC.
bd = apic.mit.polUni().fvTenant('mgmt').fvBD('inb')
bd.GET(**options.subtree)
print bd.Xml

In [ ]:
# Query all private networks.
ctxs = apic.mit.GET(**options.subtreeClass('fvCtx'))
for ctx in ctxs:
    print ctx.name

In [ ]:
# Query all private networks under management tenant.
ctxs = apic.mit.polUni().fvTenant('mgmt').GET(**options.subtreeClass('fvCtx'))
for ctx in ctxs:
    print ctx.name

In [ ]:
# Query all direct children of management tenant.
for child in apic.mit.polUni().fvTenant('mgmt').GET(**options.children):
    print child.Dn

In [ ]:
# Query entire subtree under management tenant.
for child in apic.mit.polUni().fvTenant('mgmt').GET(**options.subtree):
    print child.Dn

In [ ]:
# Query faults under out-of-band management EPG.
epg = apic.mit.polUni().fvTenant('mgmt').mgmtMgmtP('default').mgmtOoB('default')
for fault in epg.GET(**options.faults):
    print fault.descr

In [ ]:
# Query audit logs for test private network.
for change in apic.mit.polUni().fvTenant('test').fvCtx('test').GET(**options.auditLogs):
    print change.descr, change.changeSet

In [ ]:
# Use filter support to narrow query response.
for node in apic.mit.GET(**options.subtreeClass('fabricNode') &
                         options.filter(options.Eq('fabricNode.role', 'leaf') |
                                        options.Eq('fabricNode.role', 'spine'))):
    print node.name, node.role